In [3]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import datetime

In [4]:
class BarChartCreator:
    def __init__(self, df, building_no, date_range=None):
        self._df = df.copy()
        self._building_no = building_no
        self._date_range = date_range
        
        self.mark1_color = px.colors.qualitative.Plotly[1]
        self.mark2_color = px.colors.qualitative.Plotly[2]
        self.line_color = 'black'
        self.line_thick = 0.5
        self.opacity1 = 1
        self.opacity2 = 1
        
        self._fig = self.update_fig(df, building_no, date_range)
        
        
    @property
    def df(self):
        return self._df
    
    @property
    def building_no(self):
        return self._building_no
    
    @property
    def date_range(self):
        return self._date_range
    
    @property
    def fig(self):
        return self._fig
    
    def update_plot_styling(self, mark1_color=None, mark2_color=None, line_color=None, line_thick=None, opacity1=None, opacity2=None):
        if mark1_color is not None:
            self.mark1_color = mark1_color
        if mark2_color is not None:
            self.mark2_color = mark2_color
        if line_color is not None:
            self.line_color = line_color
        if line_thick is not None:
            self.line_thick = line_thick
        if opacity1 is not None:
            self.opacity1 = opacity1
        if opacity2 is not None:
            self.opacity2 = opacity2
    
    def update_fig(self, df, building_no, date_range=None):
        df = df[(df.building_no==building_no)&(df.Fan_status=='On')]
        if date_range is not None:
            df = df[(df.Datetime.dt.date>=date_range[0])&(df.Datetime.dt.date<date_range[1])]
           
        zones = df.Zone_name.unique()
         
        fig = go.Figure()
        
        for zone in zones:
            temp = df[df.Zone_name==zone]
            
            # Add main plots
            fig.add_trace(
                go.Bar(
                    x=[zone],
                    y=[temp.Fan_time_diff.sum()],
                    marker=dict(
                        color=self.mark1_color,
                        line=dict(
                            color=self.line_color,
                            width=self.line_thick
                        )
                    ),
                    opacity=self.opacity1,
                    name=zone,
                    showlegend=False,
                )
            )
            fig.add_trace(
                go.Bar(
                    x=[zone],
                    y=[temp.Predicted.sum()],
                    marker=dict(
                        color=self.mark2_color,
                        line=dict(
                            color=self.line_color,
                            width=self.line_thick
                        )
                    ),
                    opacity=self.opacity2,
                    name=zone,
                    showlegend=False,
                )
            )
        
        # Add empty plots for legend names
        fig.add_trace(
            go.Bar(x=[None],
            y=[None],
            name='Actual',
            marker=dict(color=self.mark1_color)
        ))
        fig.add_trace(
            go.Bar(x=[None],
            y=[None],
            name='Predicted',
            marker=dict(color=self.mark2_color)
        ))
            
        # Add axis titles
        fig.update_layout(
            xaxis=dict(title=f'<b>Building {self.building_no} Zones</b>'), 
            yaxis=dict(title=f'<b>Total Time Fan On (mins)</b>'), 
            barmode='overlay')
        
        return fig

In [6]:
df = pd.read_parquet(r'summary_all.parquet')

In [7]:
display(BarChartCreator(df,3).fig)

In [8]:
display(BarChartCreator(df,3).fig)

In [7]:
display(BarChartCreator(df,1,[datetime.date(2024,1,2),datetime.date(2023,12,1)]).fig)

AttributeError: Can only use .dt accessor with datetimelike values

In [38]:
import random

df.Predicted = df.Fan_time_diff.apply(lambda x: x * random.uniform(0.4, 0.95))

In [41]:
df.to_parquet(r'D:\OneDrive - Swinburne University\Comp Sci\2024 Semester 1\Group Project\Data\summary_data\summary_all.parquet', index=False)